In [1]:
"""
Creamos el dataset.
1.- Extraemos frames de un video
2.- Detectamos rostros en los frames
3.- Cramos las mascaras de los rostros
"""

'\nCreamos el dataset.\n1.- Extraemos frames de un video\n2.- Detectamos rostros en los frames\n3.- Cramos las mascaras de los rostros\n'

In [2]:
import cv2
import dlib
import numpy as np
import os
from os.path import isfile, isdir, exists

In [3]:
"""
Referencia: https://github.com/gallardorafael/edge2art
Extrae cada frame de un video y guardas las imagenes
"""
EVERY_FRAMES = 5
def extraer_frames(file_input, dir_output):
        # Validamos la existencia del video
        if isfile(file_input):
            video_path = file_input
        else:
            print("No existe el video.")
            return
        
        #Definición del CODEC
        cap = cv2.VideoCapture(file_input)
        if not exists(dir_output):
            os.mkdir(dir_output)

        # OpenCV
        contador = 0
        while cap.isOpened():
            # Obtenemos el frame.
            ret, frame = cap.read()
            if ret == True:
                if contador % EVERY_FRAMES == 0:
                    data_path = dir_output + "/" + str(contador) + ".png"
                    print("Extrayendo frame ", contador, " en ", data_path)
                    cv2.imwrite(data_path, frame)
                contador += 1
            else:
                break
        cap.release()

In [4]:
"""
OpenCV’s deep learning face detector esta basado en el
framework Single Shot Detector (SSD) con base en una red
ResNet 
"""

# Para el recorte
CROP_SIZE = 256

# Cargamos modelo de la red
path_model="./deploy.proto.txt"
path_weigths="./res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(path_model, path_weigths)
MIN_CONFIDENCE = 0.8 # Probabilidad minima para la detección

def detectar_rostro(image):
    # Cargamos la imagen y construimos una entraba para la red
    # reescalamos a 300x300 y la normalizamos
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    # Pasamos la entrada a la red y obtenemos las detecciones y 
    # predicciones
    net.setInput(blob)
    detections = net.forward()

    # Iteramos sobre las detecciones
    for i in range(0, detections.shape[2]):
        # Extraemos la probabilidad asociada a una prediccion
        confidence = detections[0, 0, i, 2]

        # Comparamos las probabilidades y verificamos que la actual 
        # sea mayor a la minima, para asegurar que sí es correcta
        # la detección
        if confidence > MIN_CONFIDENCE:
            # Calculamos las coordenadas (x, y) para el rectangulo que
            # enmarque el rostro
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            y = startY - 10 if startY - 10 > 10 else startY + 10
            distanceX = abs(endX-startX)
            distanceY = abs(endY-startY)
            startY -= distanceY * 0.2
            endY += distanceY * 0.1
            endX += distanceX * 0.5
            startX -= distanceX * 0.5
            startX = int(max(startX,0))
            startY = int(max(startY,0))
            endX = int(min(endX,w))
            endY = int(min(endY,h))
            image = image[startY:endY,startX:endX,:]
        return cv2.resize(image,(CROP_SIZE, CROP_SIZE))

![./imgs/shape_predictor_68_face_landmarks.png](./imgs/shape_predictor_68_face_landmarks.png)

In [5]:
"""
The Face Landmark Detection algorithm offered by Dlib is an
implementation of the Ensemble of Regression Trees (ERT) 
presented in 2014 by Kazemi and Sullivan. 

This technique utilize simple and fast feature (pixel 
intensities differences) to directly estimate the landmark 
positions.

These estimated positions are subsequently refined with an 
iterative process done by a cascade of regressors. The 
regressors produces a new estimate from the previous one, 
trying to reduce the alignment error of the estimated points 
at each iteration.

The algorithm is blazing fast, in fact it takes about 1–3ms
(on desktop platform) to detect (align) a set of 68 
landmarks on a given face.
"""

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")


def crear_mascara(image):
    
    try:
        image2 = image[:,:,:].copy()
    except TypeError:
        raise
        
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = cv2.merge([gray, gray, gray])
    rects = detector(gray, 1)
    
    for (i, rect) in enumerate(rects):

        if(i > 0):
            break  
        
        features = predictor(gray, rect)
        
        face = []
        eye1 = []
        eye2 = []
        eye1u = []
        eye2u = []
        nose = []
        mouth = []
        mouth2 = []
        
        for b in range(68):
            p1 = [features.part(b).x, features.part(b).y]
            
            if(b < 17):
                face.append(p1)
            
            if(b >= 22 and b <= 26):
                eye1u.append(p1) 
            
            if(b >= 17 and b <= 21):
                eye2u.append(p1)   
            
            if(b >= 36 and b <= 41):
                eye1.append(p1)
            
            if(b >= 27 and b <= 35):
                nose.append(p1)  
            
            if(b >= 42 and b <= 47):
                eye2.append(p1)
            
            if(b >= 48 and b <= 59):
                mouth.append(p1) 
            
            if(b >= 60 and b <= 68):
                mouth2.append(p1)
        
        b_image = np.zeros_like(image)
        
        cv2.fillPoly(b_image, [np.array(face)], (0,255,0))
        cv2.fillPoly(b_image, [np.array(eye1)], (255,255,0))
        cv2.fillPoly(b_image, [np.array(eye2)], (255,255,0))
        
        
        cv2.fillPoly(image, [np.array(eye1)], (255,255,0))
        cv2.fillPoly(image, [np.array(eye2)], (255,255,0))
        
        
        cv2.fillPoly(b_image, [np.array(nose)], (255,0,255))
        cv2.fillPoly(b_image, [np.array(mouth)], (0,0,0))
        cv2.fillPoly(b_image, [np.array(eye1u)], (0,255,255))
        cv2.fillPoly(b_image, [np.array(eye2u)], (0,255,255))
        cv2.fillPoly(b_image, [np.array(mouth2)],(255,0,0))
        mask = cv2.inRange(image, (255,255,0), (255,255,0))
       
        mask = cv2.bitwise_and(image2, image2, mask = mask)
        G = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask1 = cv2.threshold(G, 100, 250, cv2.THRESH_BINARY)
        mask1 = cv2.bitwise_and(255 * np.ones_like(image), 255 * np.ones_like(image),mask = mask1[1])
        mask = cv2.bitwise_or(b_image, mask1)
        
    return mask

In [6]:
def detectar_rostros_en_frames(dir_input, dir_output):
    count = 0
    # Se crea el directorio de salida
    if not isdir(dir_output):
        os.mkdir(dir_output)
    for image_name in os.listdir(dir_input):
        count += 1
        in_path = os.path.join(dir_input, image_name)
        out_path = os.path.join(dir_output, image_name)
        print("Detectando rostro ", count, " en ", in_path)
        image = cv2.imread(in_path)
        image = detectar_rostro(image)
        cv2.imwrite(in_path, image)
        print("Creando mascara ", count, " en ", out_path)
        image = crear_mascara(image)
        cv2.imwrite(out_path, image)

In [7]:
file_input = "./video1.webm"
dir_output =  "./tagged_train"
extraer_frames(file_input, dir_output)

file_input = "./video2.webm"
dir_output =  "./tagged_test"
extraer_frames(file_input, dir_output)

Extrayendo frame  0  en  ./tagged_train/0.png
Extrayendo frame  5  en  ./tagged_train/5.png
Extrayendo frame  10  en  ./tagged_train/10.png
Extrayendo frame  15  en  ./tagged_train/15.png
Extrayendo frame  20  en  ./tagged_train/20.png
Extrayendo frame  25  en  ./tagged_train/25.png
Extrayendo frame  30  en  ./tagged_train/30.png
Extrayendo frame  35  en  ./tagged_train/35.png
Extrayendo frame  40  en  ./tagged_train/40.png
Extrayendo frame  45  en  ./tagged_train/45.png
Extrayendo frame  50  en  ./tagged_train/50.png
Extrayendo frame  55  en  ./tagged_train/55.png
Extrayendo frame  60  en  ./tagged_train/60.png
Extrayendo frame  65  en  ./tagged_train/65.png
Extrayendo frame  70  en  ./tagged_train/70.png
Extrayendo frame  75  en  ./tagged_train/75.png
Extrayendo frame  80  en  ./tagged_train/80.png
Extrayendo frame  85  en  ./tagged_train/85.png
Extrayendo frame  90  en  ./tagged_train/90.png
Extrayendo frame  95  en  ./tagged_train/95.png
Extrayendo frame  100  en  ./tagged_train/10

Extrayendo frame  835  en  ./tagged_train/835.png
Extrayendo frame  840  en  ./tagged_train/840.png
Extrayendo frame  845  en  ./tagged_train/845.png
Extrayendo frame  850  en  ./tagged_train/850.png
Extrayendo frame  855  en  ./tagged_train/855.png
Extrayendo frame  860  en  ./tagged_train/860.png
Extrayendo frame  865  en  ./tagged_train/865.png
Extrayendo frame  870  en  ./tagged_train/870.png
Extrayendo frame  875  en  ./tagged_train/875.png
Extrayendo frame  880  en  ./tagged_train/880.png
Extrayendo frame  885  en  ./tagged_train/885.png
Extrayendo frame  890  en  ./tagged_train/890.png
Extrayendo frame  895  en  ./tagged_train/895.png
Extrayendo frame  900  en  ./tagged_train/900.png
Extrayendo frame  905  en  ./tagged_train/905.png
Extrayendo frame  910  en  ./tagged_train/910.png
Extrayendo frame  915  en  ./tagged_train/915.png
Extrayendo frame  920  en  ./tagged_train/920.png
Extrayendo frame  925  en  ./tagged_train/925.png
Extrayendo frame  930  en  ./tagged_train/930.png


Extrayendo frame  1630  en  ./tagged_train/1630.png
Extrayendo frame  1635  en  ./tagged_train/1635.png
Extrayendo frame  1640  en  ./tagged_train/1640.png
Extrayendo frame  1645  en  ./tagged_train/1645.png
Extrayendo frame  1650  en  ./tagged_train/1650.png
Extrayendo frame  1655  en  ./tagged_train/1655.png
Extrayendo frame  1660  en  ./tagged_train/1660.png
Extrayendo frame  1665  en  ./tagged_train/1665.png
Extrayendo frame  1670  en  ./tagged_train/1670.png
Extrayendo frame  1675  en  ./tagged_train/1675.png
Extrayendo frame  1680  en  ./tagged_train/1680.png
Extrayendo frame  1685  en  ./tagged_train/1685.png
Extrayendo frame  1690  en  ./tagged_train/1690.png
Extrayendo frame  1695  en  ./tagged_train/1695.png
Extrayendo frame  1700  en  ./tagged_train/1700.png
Extrayendo frame  1705  en  ./tagged_train/1705.png
Extrayendo frame  1710  en  ./tagged_train/1710.png
Extrayendo frame  1715  en  ./tagged_train/1715.png
Extrayendo frame  1720  en  ./tagged_train/1720.png
Extrayendo f

Extrayendo frame  2430  en  ./tagged_train/2430.png
Extrayendo frame  2435  en  ./tagged_train/2435.png
Extrayendo frame  2440  en  ./tagged_train/2440.png
Extrayendo frame  2445  en  ./tagged_train/2445.png
Extrayendo frame  2450  en  ./tagged_train/2450.png
Extrayendo frame  2455  en  ./tagged_train/2455.png
Extrayendo frame  2460  en  ./tagged_train/2460.png
Extrayendo frame  2465  en  ./tagged_train/2465.png
Extrayendo frame  2470  en  ./tagged_train/2470.png
Extrayendo frame  2475  en  ./tagged_train/2475.png
Extrayendo frame  2480  en  ./tagged_train/2480.png
Extrayendo frame  2485  en  ./tagged_train/2485.png
Extrayendo frame  2490  en  ./tagged_train/2490.png
Extrayendo frame  2495  en  ./tagged_train/2495.png
Extrayendo frame  2500  en  ./tagged_train/2500.png
Extrayendo frame  2505  en  ./tagged_train/2505.png
Extrayendo frame  2510  en  ./tagged_train/2510.png
Extrayendo frame  2515  en  ./tagged_train/2515.png
Extrayendo frame  2520  en  ./tagged_train/2520.png
Extrayendo f

Extrayendo frame  3225  en  ./tagged_train/3225.png
Extrayendo frame  3230  en  ./tagged_train/3230.png
Extrayendo frame  3235  en  ./tagged_train/3235.png
Extrayendo frame  3240  en  ./tagged_train/3240.png
Extrayendo frame  3245  en  ./tagged_train/3245.png
Extrayendo frame  3250  en  ./tagged_train/3250.png
Extrayendo frame  3255  en  ./tagged_train/3255.png
Extrayendo frame  3260  en  ./tagged_train/3260.png
Extrayendo frame  3265  en  ./tagged_train/3265.png
Extrayendo frame  3270  en  ./tagged_train/3270.png
Extrayendo frame  3275  en  ./tagged_train/3275.png
Extrayendo frame  3280  en  ./tagged_train/3280.png
Extrayendo frame  3285  en  ./tagged_train/3285.png
Extrayendo frame  3290  en  ./tagged_train/3290.png
Extrayendo frame  3295  en  ./tagged_train/3295.png
Extrayendo frame  3300  en  ./tagged_train/3300.png
Extrayendo frame  3305  en  ./tagged_train/3305.png
Extrayendo frame  3310  en  ./tagged_train/3310.png
Extrayendo frame  3315  en  ./tagged_train/3315.png
Extrayendo f

...

Extrayendo frame  12680  en  ./tagged_test/12680.png
Extrayendo frame  12685  en  ./tagged_test/12685.png
Extrayendo frame  12690  en  ./tagged_test/12690.png
Extrayendo frame  12695  en  ./tagged_test/12695.png
Extrayendo frame  12700  en  ./tagged_test/12700.png
Extrayendo frame  12705  en  ./tagged_test/12705.png
Extrayendo frame  12710  en  ./tagged_test/12710.png
Extrayendo frame  12715  en  ./tagged_test/12715.png
Extrayendo frame  12720  en  ./tagged_test/12720.png
Extrayendo frame  12725  en  ./tagged_test/12725.png
Extrayendo frame  12730  en  ./tagged_test/12730.png
Extrayendo frame  12735  en  ./tagged_test/12735.png
Extrayendo frame  12740  en  ./tagged_test/12740.png
Extrayendo frame  12745  en  ./tagged_test/12745.png
Extrayendo frame  12750  en  ./tagged_test/12750.png
Extrayendo frame  12755  en  ./tagged_test/12755.png
Extrayendo frame  12760  en  ./tagged_test/12760.png
Extrayendo frame  12765  en  ./tagged_test/12765.png
Extrayendo frame  12770  en  ./tagged_test/127

Extrayendo frame  13455  en  ./tagged_test/13455.png
Extrayendo frame  13460  en  ./tagged_test/13460.png
Extrayendo frame  13465  en  ./tagged_test/13465.png
Extrayendo frame  13470  en  ./tagged_test/13470.png
Extrayendo frame  13475  en  ./tagged_test/13475.png
Extrayendo frame  13480  en  ./tagged_test/13480.png
Extrayendo frame  13485  en  ./tagged_test/13485.png
Extrayendo frame  13490  en  ./tagged_test/13490.png
Extrayendo frame  13495  en  ./tagged_test/13495.png
Extrayendo frame  13500  en  ./tagged_test/13500.png
Extrayendo frame  13505  en  ./tagged_test/13505.png
Extrayendo frame  13510  en  ./tagged_test/13510.png
Extrayendo frame  13515  en  ./tagged_test/13515.png
Extrayendo frame  13520  en  ./tagged_test/13520.png
Extrayendo frame  13525  en  ./tagged_test/13525.png
Extrayendo frame  13530  en  ./tagged_test/13530.png
Extrayendo frame  13535  en  ./tagged_test/13535.png
Extrayendo frame  13540  en  ./tagged_test/13540.png
Extrayendo frame  13545  en  ./tagged_test/135

Extrayendo frame  14230  en  ./tagged_test/14230.png
Extrayendo frame  14235  en  ./tagged_test/14235.png
Extrayendo frame  14240  en  ./tagged_test/14240.png
Extrayendo frame  14245  en  ./tagged_test/14245.png
Extrayendo frame  14250  en  ./tagged_test/14250.png
Extrayendo frame  14255  en  ./tagged_test/14255.png
Extrayendo frame  14260  en  ./tagged_test/14260.png
Extrayendo frame  14265  en  ./tagged_test/14265.png
Extrayendo frame  14270  en  ./tagged_test/14270.png
Extrayendo frame  14275  en  ./tagged_test/14275.png
Extrayendo frame  14280  en  ./tagged_test/14280.png
Extrayendo frame  14285  en  ./tagged_test/14285.png
Extrayendo frame  14290  en  ./tagged_test/14290.png
Extrayendo frame  14295  en  ./tagged_test/14295.png
Extrayendo frame  14300  en  ./tagged_test/14300.png
Extrayendo frame  14305  en  ./tagged_test/14305.png
Extrayendo frame  14310  en  ./tagged_test/14310.png
Extrayendo frame  14315  en  ./tagged_test/14315.png
Extrayendo frame  14320  en  ./tagged_test/143

Extrayendo frame  15005  en  ./tagged_test/15005.png
Extrayendo frame  15010  en  ./tagged_test/15010.png
Extrayendo frame  15015  en  ./tagged_test/15015.png
Extrayendo frame  15020  en  ./tagged_test/15020.png
Extrayendo frame  15025  en  ./tagged_test/15025.png
Extrayendo frame  15030  en  ./tagged_test/15030.png
Extrayendo frame  15035  en  ./tagged_test/15035.png
Extrayendo frame  15040  en  ./tagged_test/15040.png
Extrayendo frame  15045  en  ./tagged_test/15045.png
Extrayendo frame  15050  en  ./tagged_test/15050.png
Extrayendo frame  15055  en  ./tagged_test/15055.png
Extrayendo frame  15060  en  ./tagged_test/15060.png
Extrayendo frame  15065  en  ./tagged_test/15065.png
Extrayendo frame  15070  en  ./tagged_test/15070.png
Extrayendo frame  15075  en  ./tagged_test/15075.png
Extrayendo frame  15080  en  ./tagged_test/15080.png
Extrayendo frame  15085  en  ./tagged_test/15085.png
Extrayendo frame  15090  en  ./tagged_test/15090.png
Extrayendo frame  15095  en  ./tagged_test/150

Extrayendo frame  15790  en  ./tagged_test/15790.png
Extrayendo frame  15795  en  ./tagged_test/15795.png
Extrayendo frame  15800  en  ./tagged_test/15800.png
Extrayendo frame  15805  en  ./tagged_test/15805.png
Extrayendo frame  15810  en  ./tagged_test/15810.png
Extrayendo frame  15815  en  ./tagged_test/15815.png
Extrayendo frame  15820  en  ./tagged_test/15820.png
Extrayendo frame  15825  en  ./tagged_test/15825.png
Extrayendo frame  15830  en  ./tagged_test/15830.png
Extrayendo frame  15835  en  ./tagged_test/15835.png
Extrayendo frame  15840  en  ./tagged_test/15840.png
Extrayendo frame  15845  en  ./tagged_test/15845.png
Extrayendo frame  15850  en  ./tagged_test/15850.png
Extrayendo frame  15855  en  ./tagged_test/15855.png
Extrayendo frame  15860  en  ./tagged_test/15860.png
Extrayendo frame  15865  en  ./tagged_test/15865.png
Extrayendo frame  15870  en  ./tagged_test/15870.png
Extrayendo frame  15875  en  ./tagged_test/15875.png
Extrayendo frame  15880  en  ./tagged_test/158

In [8]:
dir_input = "./tagged_train"
dir_output =  "./input_train"
detectar_rostros_en_frames(dir_input, dir_output)

dir_input = "./tagged_test"
dir_output =  "./input_test"
detectar_rostros_en_frames(dir_input, dir_output)

Detectando rostro  1  en  ./tagged_train/210.png
Creando mascara  1  en  ./input_train/210.png
Detectando rostro  2  en  ./tagged_train/525.png
Creando mascara  2  en  ./input_train/525.png
Detectando rostro  3  en  ./tagged_train/6095.png
Creando mascara  3  en  ./input_train/6095.png
Detectando rostro  4  en  ./tagged_train/7580.png
Creando mascara  4  en  ./input_train/7580.png
Detectando rostro  5  en  ./tagged_train/7790.png
Creando mascara  5  en  ./input_train/7790.png
Detectando rostro  6  en  ./tagged_train/295.png
Creando mascara  6  en  ./input_train/295.png
Detectando rostro  7  en  ./tagged_train/6315.png
Creando mascara  7  en  ./input_train/6315.png
Detectando rostro  8  en  ./tagged_train/5115.png
Creando mascara  8  en  ./input_train/5115.png
Detectando rostro  9  en  ./tagged_train/2855.png
Creando mascara  9  en  ./input_train/2855.png
Detectando rostro  10  en  ./tagged_train/7920.png
Creando mascara  10  en  ./input_train/7920.png
Detectando rostro  11  en  ./tagge

Creando mascara  84  en  ./input_train/9450.png
Detectando rostro  85  en  ./tagged_train/990.png
Creando mascara  85  en  ./input_train/990.png
Detectando rostro  86  en  ./tagged_train/2110.png
Creando mascara  86  en  ./input_train/2110.png
Detectando rostro  87  en  ./tagged_train/2425.png
Creando mascara  87  en  ./input_train/2425.png
Detectando rostro  88  en  ./tagged_train/1965.png
Creando mascara  88  en  ./input_train/1965.png
Detectando rostro  89  en  ./tagged_train/6040.png
Creando mascara  89  en  ./input_train/6040.png
Detectando rostro  90  en  ./tagged_train/2600.png
Creando mascara  90  en  ./input_train/2600.png
Detectando rostro  91  en  ./tagged_train/9385.png
Creando mascara  91  en  ./input_train/9385.png
Detectando rostro  92  en  ./tagged_train/1110.png
Creando mascara  92  en  ./input_train/1110.png
Detectando rostro  93  en  ./tagged_train/9045.png
Creando mascara  93  en  ./input_train/9045.png
Detectando rostro  94  en  ./tagged_train/975.png
Creando masca

Creando mascara  166  en  ./input_train/1255.png
Detectando rostro  167  en  ./tagged_train/1775.png
Creando mascara  167  en  ./input_train/1775.png
Detectando rostro  168  en  ./tagged_train/9130.png
Creando mascara  168  en  ./input_train/9130.png
Detectando rostro  169  en  ./tagged_train/435.png
Creando mascara  169  en  ./input_train/435.png
Detectando rostro  170  en  ./tagged_train/4970.png
Creando mascara  170  en  ./input_train/4970.png
Detectando rostro  171  en  ./tagged_train/9205.png
Creando mascara  171  en  ./input_train/9205.png
Detectando rostro  172  en  ./tagged_train/4640.png
Creando mascara  172  en  ./input_train/4640.png
Detectando rostro  173  en  ./tagged_train/5695.png
Creando mascara  173  en  ./input_train/5695.png
Detectando rostro  174  en  ./tagged_train/2005.png
Creando mascara  174  en  ./input_train/2005.png
Detectando rostro  175  en  ./tagged_train/6130.png
Creando mascara  175  en  ./input_train/6130.png
Detectando rostro  176  en  ./tagged_train/7

Creando mascara  249  en  ./input_train/5250.png
Detectando rostro  250  en  ./tagged_train/2720.png
Creando mascara  250  en  ./input_train/2720.png
Detectando rostro  251  en  ./tagged_train/835.png
Creando mascara  251  en  ./input_train/835.png
Detectando rostro  252  en  ./tagged_train/6005.png
Creando mascara  252  en  ./input_train/6005.png
Detectando rostro  253  en  ./tagged_train/200.png
Creando mascara  253  en  ./input_train/200.png
Detectando rostro  254  en  ./tagged_train/1230.png
Creando mascara  254  en  ./input_train/1230.png
Detectando rostro  255  en  ./tagged_train/7685.png
Creando mascara  255  en  ./input_train/7685.png
Detectando rostro  256  en  ./tagged_train/3855.png
Creando mascara  256  en  ./input_train/3855.png
Detectando rostro  257  en  ./tagged_train/5875.png
Creando mascara  257  en  ./input_train/5875.png
Detectando rostro  258  en  ./tagged_train/2105.png
Creando mascara  258  en  ./input_train/2105.png
Detectando rostro  259  en  ./tagged_train/540

Creando mascara  331  en  ./input_train/2375.png
Detectando rostro  332  en  ./tagged_train/9350.png
Creando mascara  332  en  ./input_train/9350.png
Detectando rostro  333  en  ./tagged_train/4390.png
Creando mascara  333  en  ./input_train/4390.png
Detectando rostro  334  en  ./tagged_train/5840.png
Creando mascara  334  en  ./input_train/5840.png
Detectando rostro  335  en  ./tagged_train/3025.png
Creando mascara  335  en  ./input_train/3025.png
Detectando rostro  336  en  ./tagged_train/5520.png
Creando mascara  336  en  ./input_train/5520.png
Detectando rostro  337  en  ./tagged_train/7815.png
Creando mascara  337  en  ./input_train/7815.png
Detectando rostro  338  en  ./tagged_train/95.png
Creando mascara  338  en  ./input_train/95.png
Detectando rostro  339  en  ./tagged_train/2560.png
Creando mascara  339  en  ./input_train/2560.png
Detectando rostro  340  en  ./tagged_train/3000.png
Creando mascara  340  en  ./input_train/3000.png
Detectando rostro  341  en  ./tagged_train/247

...

Creando mascara  444  en  ./input_test/905.png
Detectando rostro  445  en  ./tagged_test/275.png
Creando mascara  445  en  ./input_test/275.png
Detectando rostro  446  en  ./tagged_test/4335.png
Creando mascara  446  en  ./input_test/4335.png
Detectando rostro  447  en  ./tagged_test/14160.png
Creando mascara  447  en  ./input_test/14160.png
Detectando rostro  448  en  ./tagged_test/6890.png
Creando mascara  448  en  ./input_test/6890.png
Detectando rostro  449  en  ./tagged_test/1455.png
Creando mascara  449  en  ./input_test/1455.png
Detectando rostro  450  en  ./tagged_test/320.png
Creando mascara  450  en  ./input_test/320.png
Detectando rostro  451  en  ./tagged_test/8090.png
Creando mascara  451  en  ./input_test/8090.png
Detectando rostro  452  en  ./tagged_test/1345.png
Creando mascara  452  en  ./input_test/1345.png
Detectando rostro  453  en  ./tagged_test/8490.png
Creando mascara  453  en  ./input_test/8490.png
Detectando rostro  454  en  ./tagged_test/5900.png
Creando masca

Detectando rostro  528  en  ./tagged_test/1555.png
Creando mascara  528  en  ./input_test/1555.png
Detectando rostro  529  en  ./tagged_test/14205.png
Creando mascara  529  en  ./input_test/14205.png
Detectando rostro  530  en  ./tagged_test/7715.png
Creando mascara  530  en  ./input_test/7715.png
Detectando rostro  531  en  ./tagged_test/3420.png
Creando mascara  531  en  ./input_test/3420.png
Detectando rostro  532  en  ./tagged_test/920.png
Creando mascara  532  en  ./input_test/920.png
Detectando rostro  533  en  ./tagged_test/14390.png
Creando mascara  533  en  ./input_test/14390.png
Detectando rostro  534  en  ./tagged_test/14440.png
Creando mascara  534  en  ./input_test/14440.png
Detectando rostro  535  en  ./tagged_test/4195.png
Creando mascara  535  en  ./input_test/4195.png
Detectando rostro  536  en  ./tagged_test/3015.png
Creando mascara  536  en  ./input_test/3015.png
Detectando rostro  537  en  ./tagged_test/345.png
Creando mascara  537  en  ./input_test/345.png
Detectan

Creando mascara  611  en  ./input_test/13020.png
Detectando rostro  612  en  ./tagged_test/3875.png
Creando mascara  612  en  ./input_test/3875.png
Detectando rostro  613  en  ./tagged_test/8855.png
Creando mascara  613  en  ./input_test/8855.png
Detectando rostro  614  en  ./tagged_test/1500.png
Creando mascara  614  en  ./input_test/1500.png
Detectando rostro  615  en  ./tagged_test/7655.png
Creando mascara  615  en  ./input_test/7655.png
Detectando rostro  616  en  ./tagged_test/3870.png
Creando mascara  616  en  ./input_test/3870.png
Detectando rostro  617  en  ./tagged_test/8415.png
Creando mascara  617  en  ./input_test/8415.png
Detectando rostro  618  en  ./tagged_test/6910.png
Creando mascara  618  en  ./input_test/6910.png
Detectando rostro  619  en  ./tagged_test/6735.png
Creando mascara  619  en  ./input_test/6735.png
Detectando rostro  620  en  ./tagged_test/1530.png
Creando mascara  620  en  ./input_test/1530.png
Detectando rostro  621  en  ./tagged_test/5860.png
Creando m

Creando mascara  694  en  ./input_test/8110.png
Detectando rostro  695  en  ./tagged_test/5815.png
Creando mascara  695  en  ./input_test/5815.png
Detectando rostro  696  en  ./tagged_test/12870.png
Creando mascara  696  en  ./input_test/12870.png
Detectando rostro  697  en  ./tagged_test/8805.png
Creando mascara  697  en  ./input_test/8805.png
Detectando rostro  698  en  ./tagged_test/1430.png
Creando mascara  698  en  ./input_test/1430.png
Detectando rostro  699  en  ./tagged_test/6030.png
Creando mascara  699  en  ./input_test/6030.png
Detectando rostro  700  en  ./tagged_test/6045.png
Creando mascara  700  en  ./input_test/6045.png
Detectando rostro  701  en  ./tagged_test/14340.png
Creando mascara  701  en  ./input_test/14340.png
Detectando rostro  702  en  ./tagged_test/13850.png
Creando mascara  702  en  ./input_test/13850.png
Detectando rostro  703  en  ./tagged_test/1290.png
Creando mascara  703  en  ./input_test/1290.png
Detectando rostro  704  en  ./tagged_test/7660.png
Crea

Creando mascara  778  en  ./input_test/7540.png
Detectando rostro  779  en  ./tagged_test/1910.png
Creando mascara  779  en  ./input_test/1910.png
Detectando rostro  780  en  ./tagged_test/13920.png
Creando mascara  780  en  ./input_test/13920.png
Detectando rostro  781  en  ./tagged_test/7625.png
Creando mascara  781  en  ./input_test/7625.png
Detectando rostro  782  en  ./tagged_test/2780.png
Creando mascara  782  en  ./input_test/2780.png
Detectando rostro  783  en  ./tagged_test/11705.png
Creando mascara  783  en  ./input_test/11705.png
Detectando rostro  784  en  ./tagged_test/660.png
Creando mascara  784  en  ./input_test/660.png
Detectando rostro  785  en  ./tagged_test/2245.png
Creando mascara  785  en  ./input_test/2245.png
Detectando rostro  786  en  ./tagged_test/3520.png
Creando mascara  786  en  ./input_test/3520.png
Detectando rostro  787  en  ./tagged_test/1520.png
Creando mascara  787  en  ./input_test/1520.png
Detectando rostro  788  en  ./tagged_test/5810.png
Creando 